# Using Preview Simulators with Q# and Python

The preview open systems and stabilizer simulators for the Quantum Development Kit use the [QuTiP](https://qutip.org) library for Python to help represent noise models, so we import it here.

In [1]:
import qutip as qt

To use the preview simulators, we start by importing Q# interoperability as normal.

In [2]:
import qsharp

We can then use `qsharp.experimental.enable_noisy_simulation()` to add support for experimental simulators.

In [3]:
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()

Doing so adds the `.simulate_noise` method to Python representations of Q# callables:

In [4]:
%%qsharp

operation DumpPlus() : Unit {
    use q = Qubit();
    H(q);
    Microsoft.Quantum.Diagnostics.DumpMachine();
    X(q);
    Reset(q);
}

In [5]:
DumpPlus.simulate_noise()

# of qubits,1
State data,$$ \left( \begin{matrix} 0.5000000000000001 + 0 i & 0.5000000000000001 + 0 i\\ 0.5000000000000001 + 0 i & 0.5000000000000001 + 0 i \end{matrix} \right) $$


()

Looking at the output from the above, we notice two distinct differences with the output from `.simulate()`:

- The preview simulators use quantum registers of a fixed size (by default, three qubits), and allocate qubits from that register.
- By default, the preview simulators represent quantum states as density operators ($\rho = \left|\psi\right\rangle\left\langle\psi\right|$) instead of as state vectors ($\left|\psi\right\rangle$).

For example, in the output above, the preview simulator has output the density operator $\rho = \left|+00\right\rangle\left\langle+00\right|$, as we can verify by using QuTiP.

In [6]:
ket_zero = qt.basis(2, 0)
ket_zero

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]

In [7]:
ket_one = qt.basis(2, 1)
ket_plus = (ket_zero + ket_one).unit()
ket_plus

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

In [8]:
ket_psi = qt.tensor(ket_plus, ket_zero, ket_zero)
rho = ket_psi * ket_psi.dag()
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]

## Configuring Open Systems Noise Models

The preview simulators can be configured by the use of the `qsharp.config` object. For example, to change the size of the register used, we can modify the `experimental.simulators.nQubits` configuration setting:

In [9]:
qsharp.config['experimental.simulators.nQubits'] = 1

In [10]:
DumpPlus.simulate_noise()

# of qubits,1
State data,$$ \left( \begin{matrix} 0.5000000000000001 + 0 i & 0.5000000000000001 + 0 i\\ 0.5000000000000001 + 0 i & 0.5000000000000001 + 0 i \end{matrix} \right) $$


()

We can modify the noise model used in simulating Q# programs by using several functions in the `qsharp.experimental` module. For instance, to initialize the noise model to an ideal model (that is, with no noise), we can use `set_noise_model_by_name` or the `%noise_model --set-by-name` magic command:

In [11]:
qsharp.experimental.set_noise_model_by_name('ideal')
%noise_model --set-by-name ideal

We can then access the noise model by using `get_noise_model`:

In [12]:
noise_model = qsharp.experimental.get_noise_model()

This noise model is represented as a Python dictionary from preparations, measurements, and gates to Python objects representing the noise in each. For example, in the ideal noise model, the `Microsoft.Quantum.Intrinsic.H` operation is simulated by a unitary matrix:

In [13]:
noise_model['h']

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

We can modify this to add depolarizing noise using QuTiP functions to build a depolarizing noise channel:

In [14]:
I, X, Y, Z = [P.as_qobj() for P in qsharp.Pauli]

In [15]:
def depolarizing_noise(p=1.0):
    return p * qt.to_super(I) + ((1 - p) / 4) * sum(map(qt.to_super, [I, X, Y, Z]))

In [16]:
noise_model['h'] = depolarizing_noise(0.99) * qt.to_super(qt.qip.operations.hadamard_transform())
noise_model['h']

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True
Qobj data =
[[ 0.5    0.495  0.495  0.5  ]
 [ 0.495 -0.495  0.495 -0.495]
 [ 0.495  0.495 -0.495 -0.495]
 [ 0.5   -0.495 -0.495  0.5  ]]

In [17]:
ket_zero = qt.basis(2, 0)
ket_zero

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[1.]
 [0.]]

In [18]:
rho_zero = ket_zero * ket_zero.dag()
rho_zero

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [19]:
noise_model['h'](rho_zero)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5   0.495]
 [0.495 0.5  ]]

Once we have modified our noise model in this way, we can set it as the active noise model used in simulating Q# programs:

In [20]:
qsharp.experimental.set_noise_model(noise_model)

Using this model, we no longer get the exact $|+\rangle\langle+|$ state, but see that our Q# program has incurred some small error due to noise in the application of `Microsoft.Quantum.Intrinsic.H`:

In [21]:
DumpPlus.simulate_noise()

# of qubits,1
State data,$$ \left( \begin{matrix} 0.5032581095356969 + 0 i & 0.4951069263733158 + 0 i\\ 0.4951069263733158 + 0 i & 0.49667422634133085 + 0 i \end{matrix} \right) $$


()

In [22]:
qt.to_kraus(noise_model['h'])

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[ 0.06123724 -0.02041241]
  [-0.02041241  0.02041241]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[ 0.70445014  0.70445014]
  [ 0.70445014 -0.70445014]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
 Qobj data =
 [[ 0.05707046 -0.02948997]
  [ 0.00190948  0.02948997]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
 Qobj data =
 [[-0.00103545  0.04950143]
  [ 0.00197827  0.05044425]]]

## Configuring Stabilizer Noise Models

We can also configure the preview simulator to use stabilizer (_a.k.a._ CHP) simulation. This time, let's get a new noise model by using `get_noise_model_by_name`:

In [23]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal_stabilizer')
noise_model

{'initial_state': {'n_qubits': 1,
  'data': {'Stabilizer': {'n_qubits': 1,
    'table': array([[ True, False, False],
           [False,  True, False]])}}},
 'cnot': {'n_qubits': 2, 'data': {'ChpDecomposition': [{'Cnot': [0, 1]}]}},
 'i': {'n_qubits': 1, 'data': {'Sequence': []}},
 's': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'Phase': 0}]}},
 's_adj': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'AdjointPhase': 0}]}},
 't': {'n_qubits': 1, 'data': 'Unsupported'},
 't_adj': {'n_qubits': 1, 'data': 'Unsupported'},
 'h': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'Hadamard': 0}]}},
 'x': {'n_qubits': 1,
  'data': {'ChpDecomposition': [{'Hadamard': 0},
    {'Phase': 0},
    {'Phase': 0},
    {'Hadamard': 0}]}},
 'y': {'n_qubits': 1,
  'data': {'ChpDecomposition': [{'AdjointPhase': 0},
    {'Hadamard': 0},
    {'Phase': 0},
    {'Phase': 0},
    {'Hadamard': 0},
    {'Phase': 0}]}},
 'z': {'n_qubits': 1,
  'data': {'ChpDecomposition': [{'Phase': 0}, {'Phase': 0}]}},
 'z_meas': 

In [24]:
qsharp.experimental.set_noise_model(noise_model)

To make the best use of stabilizer noise models, we also need to configure the simulator to start off in the stabilizer representation:

In [25]:
qsharp.config['experimental.simulators.representation'] = 'stabilizer'

In [26]:
DumpPlus.simulate_noise()

# of qubits,1
State data,$$\left\langle X \right\rangle$$


()

Notably, the stabilizer representation does not support operations outside of the stabilizer formalism, such as `T` and `CCNOT`. This allows the stabilizer representation to support significantly more qubits than other representations:

In [27]:
qsharp.config['experimental.simulators.nQubits'] = 10

In [28]:
DumpPlus.simulate_noise()

# of qubits,10
State data,"$$\left\langle X𝟙𝟙𝟙𝟙𝟙𝟙𝟙𝟙𝟙, 𝟙Z𝟙𝟙𝟙𝟙𝟙𝟙𝟙𝟙, 𝟙𝟙Z𝟙𝟙𝟙𝟙𝟙𝟙𝟙, 𝟙𝟙𝟙Z𝟙𝟙𝟙𝟙𝟙𝟙, 𝟙𝟙𝟙𝟙Z𝟙𝟙𝟙𝟙𝟙, 𝟙𝟙𝟙𝟙𝟙Z𝟙𝟙𝟙𝟙, 𝟙𝟙𝟙𝟙𝟙𝟙Z𝟙𝟙𝟙, 𝟙𝟙𝟙𝟙𝟙𝟙𝟙Z𝟙𝟙, 𝟙𝟙𝟙𝟙𝟙𝟙𝟙𝟙Z𝟙, 𝟙𝟙𝟙𝟙𝟙𝟙𝟙𝟙𝟙Z \right\rangle$$"


()

If we turn off visualization, we can get significantly more qubits still!

In [29]:
%%qsharp
open Microsoft.Quantum.Arrays;

operation SampleRandomBitstring(nQubits : Int) : Result[] {
    use register = Qubit[nQubits];
    ApplyToEachCA(H, register);
    return ForEach(M, register);
}

In [30]:
qsharp.config['experimental.simulators.nQubits'] = 1000

In [31]:
%time SampleRandomBitstring.simulate_noise(nQubits=1000)

Wall time: 2.79 s


[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,


For now, though, we'll turn back down the number of qubits just to make dumps easier to read!

In [32]:
qsharp.config['experimental.simulators.nQubits'] = 4

The visualization style for stabilizer states can be selected by using the `experimental.simulators.stabilizerStateStyle` configuration setting:

In [33]:
%%qsharp

operation DumpBellPair() : Unit {
    use left = Qubit();
    use right = Qubit();
    within {
        H(left);
        CNOT(left, right);
    } apply {
        Microsoft.Quantum.Diagnostics.DumpMachine();
    }
}

In [34]:
qsharp.config['experimental.simulators.stabilizerStateStyle'] = 'matrixWithoutDestabilizers'

In [35]:
DumpBellPair.simulate_noise()

# of qubits,4
State data,$$\left(\begin{array}{cccc|cccc|c}1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 & 0\\ 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\end{array}\right)$$


()

In [36]:
DumpBellPair.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

()

In [37]:
qsharp.config['experimental.simulators.stabilizerStateStyle'] = 'denseGroupPresentation'

In [38]:
DumpBellPair.simulate_noise()

# of qubits,4
State data,"$$\left\langle XX𝟙𝟙, ZZ𝟙𝟙, 𝟙𝟙Z𝟙, 𝟙𝟙𝟙Z \right\rangle$$"


()

In [39]:
qsharp.config['experimental.simulators.stabilizerStateStyle'] = 'sparseGroupPresentation'

In [40]:
DumpBellPair.simulate_noise()

# of qubits,4
State data,"$$\left\langle X_{0}X_{1}, Z_{0}Z_{1}, Z_{2}, Z_{3} \right\rangle$$"


()

So far, we've only used ideal stabilizer simulation, but what happens if one of our operations is followed by a mixed Pauli channel?

In [41]:
noise_model['h'] = qsharp.experimental.SequenceProcess(1, 
    [
        qsharp.experimental.ChpDecompositionProcess(1, [
            qsharp.experimental.Hadamard(0)
        ]),
        qsharp.experimental.MixedPauliProcess(1, [
            (0.9, 'I'),
            (0.1, 'Z')
        ])
    ]
)
noise_model['h']

SequenceProcess(n_qubits=1, processes=[ChpDecompositionProcess(n_qubits=1, operations=[Hadamard(idx_target=0)]), MixedPauliProcess(n_qubits=1, operators=[(0.9, 'I'), (0.1, 'Z')])])

In [42]:
qsharp.experimental.set_noise_model(noise_model)

In [43]:
DumpBellPair.simulate_noise()

# of qubits,4
State data,"$$\left\langle X_{0}X_{1}, Z_{0}Z_{1}, Z_{2}, Z_{3} \right\rangle$$"


()

## Epilogue

In [44]:
qsharp.component_versions()

{'iqsharp': LooseVersion ('0.17.210628040-alpha'),
 'Jupyter Core': LooseVersion ('1.5.0.0'),
 '.NET Runtime': LooseVersion ('.NETCoreApp,Version=v3.1'),
 'qsharp': LooseVersion ('0.17.2106.27950a1'),
 'experimental': {'simulators': {'features': ['DEFAULT'],
   'name': 'Microsoft.Quantum.Experimental.Simulators',
   'opt_level': '3',
   'target': 'x86_64-pc-windows-msvc',
   'version': '0.17.210628040-alpha'}}}